In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import datetime

In [ ]:
#### Converting pre-processed images to NumPy arrays

# List full paths of all files in the directory
image_dir = "data/train/images"
image_paths = [os.path.join(image_dir, "post/imag", image_name) for image_name in sorted(os.listdir(os.path.join(image_dir, "post/imag"))) 
             if image_name.endswith('.nii')]

# Convert sitk images to arrays
image_arrays = []
for image_path in image_paths:
    image = sitk.ReadImage(image_path)
    image_array = sitk.GetArrayFromImage(image)
    image_arrays.append(image_array)
    print(f"Image array created for {image_path[28:]}")

# Convert the list of image arrays to a single NumPy array
# Make sure that all the images have the same dimensions before converting them to a NumPy array.
x_train = np.array(image_arrays)
print(f"x_train created with shape {x_train.shape}")

In [ ]:
#### Converting pre-processed masks to NumPy arrays

# List full paths of all files in the directory
image_dir = "data/train/images"
mask_paths = [os.path.join(image_dir, "post/mask", mask_name) for mask_name in sorted(os.listdir(os.path.join(image_dir, "post/mask"))) 
             if mask_name.endswith('.nii')]

# Convert sitk images to arrays
mask_arrays = []
for mask_path in mask_paths:
    mask = sitk.ReadImage(mask_path)
    mask_array = sitk.GetArrayFromImage(image)
    mask_arrays.append(mask_array)
    print(f"Image array created for {mask_path[28:]}")

# Convert the list of image arrays to a single NumPy array
# Make sure that all the images have the same dimensions before converting them to a NumPy array.
y_train = np.array(mask_arrays)
print(f"y_train created with shape {y_train.shape}")

In [ ]:
from keras import backend as K
from resnet3d import Resnet3DBuilder # pip install git+https://github.com/JihongJu/keras-resnet3d.git

bchs = 4
epch = 150
file = 'simple_model'

ir = x_train.shape[1]
ic = x_train.shape[2]
ih = x_train.shape[3]
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, ir, ic, ih)
    # x_test = x_test.reshape(x_test.shape[0], 1, ir, ic, ih)
    input_shape = (1, ir, ic, ih)
else:
    x_train = x_train.reshape(x_train.shape[0], ir, ic, ih, 1)
    # x_test = x_test.reshape(x_test.shape[0], ir, ic, ih, 1)
    input_shape = (ir, ic, ih, 1)

nb_classes = y_train.shape[1] # negative number sets regression
    
model = Resnet3DBuilder.build_resnet_50(input_shape, nb_classes)
model.compile(optimizer = 'adam',
              loss = 'mean_squared_error', 
              metrics = ['mae'])

trainstarttime = datetime.datetime.today()
print("Train start time" + str(trainstarttime))

history = model.fit(x_train, y_train,
            batch_size = bchs,
            epochs = epch,
            verbose = 1) # validation_data = (x_test, y_test)

trainfinishtime = datetime.datetime.today()
print("Train finish time" + str(trainfinishtime))
ttime = str(trainfinishtime - trainstarttime)
print("training time" + ttime)

json_string = model.to_json()
fj = file + '.json'
fh = file + '.h5'
open(fj, 'w').write(json_string)
model.save_weights(fh)